run block with #execute to execute certain function
run block with #check to check the certain property 

In [22]:
#execute
# add the path and load the module
#(I haven't write the setup.py)
import os 
os.chdir('/home/zhenfeng/ProteinFeature_1/DPMacro/tutorial_AFill')
import sys
sys.path.append('/home/zhenfeng/ProteinFeature_1/DPMacro')
import AFill as af


In [2]:
#check the docstring for full help doc 
help(af)

Help on module AFill:

NAME
    AFill

CLASSES
    BaseClasses.ResidueFeatureExtractor(BaseClasses.StructFeatureExtractor)
        structure_aligner
    
    class structure_aligner(BaseClasses.ResidueFeatureExtractor)
     |  structure_aligner(rna: bool = False)
     |  
     |  Method resolution order:
     |      structure_aligner
     |      BaseClasses.ResidueFeatureExtractor
     |      BaseClasses.StructFeatureExtractor
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __init__(self, rna: bool = False)
     |      set rna=True if processing RNA
     |      so far it only affects the generation of self.sequences and self.reference_sequences
     |  
     |  align_sequence(self, object_chain: str, reference_chain: str) -> pandas.core.frame.DataFrame
     |      align two sequences, one from object structure and another from reference structure
     |      
     |      1.set self.alignment._records, a MultipleSeqAlignment instance.
     |      2.se

In [3]:
#execute
#load the model(object) structure and the rev structure 
aligner=af.structure_aligner()
aligner.load_structure('multimer_af2_afm_ft_0506_c16_sd0.5_s512_4k.pt_9871_r11_e1.pdb')
aligner.load_reference('1h2r.pdb')

In [4]:
#check
#check the fasta information
aligner.sequences
aligner.reference_sequences

{'S': 'LMGPRRPSVVYLHNAECTGCSESVLRAFEPYIDTLILDTLSLDYHETIMAAAGDAAEAALEQAVNSPHGFIAVVEGGIPTAANGIYGKVANHTMLDICSRILPKAQAVIAYGTCATFGGVQAAKPNPTGAKGVNDALKHLGVKAINIAGCPPNPYNLVGTIVYYLKNKAAPELDSLNRPTMFFGQTVHEQCPRLPHFDAGEFAPSFESEEARKGWCLYELGCKGPVTMNNCPKIKFNQTNWPVDAGHPCIGCSEPDFWDAMTPFYQN',
 'L': 'SSYSGPIVVDPVTRIEGHLRIEVEVENGKVKNAYSSSTLFRGLEIILKGRDPRDAQHFTQRTCGVCTYTHALASTRCVDNAVGVHIPKNATYIRNLVLGAQYLHDHIVHFYHLHALDFVDVTAALKADPAKAAKVASSISPRKTTAADLKAVQDKLKTFVETGQLGPFTNAYFLGGHPAYYLDPETNLIATAHYLEALRLQVKAARAMAVFGAKNPHTQFTVVGGVTCYDALTPQRIAEFEALWKETKAFVDEVYIPDLLVVAAAYKDWTQYGGTDNFITFGEFPKDEYDLNSRFFKPGVVFKRDFKNIKPFDKMQIEEHVRHSWYEGAEARHPWKGQTQPKYTDLHGDDRYSWMKAPRYMGEPMETGPLAQVLIAYSQGHPKVKAVTDAVLAKLGVGPEALFSTLGRTAARGIETAVIAEYVGVMLQEYKDNIAKGDNVICAPWEMPKQAEGVGFVNAPRGGLSHWIRIEDGKIGNFQLVVPSTWTLGPRCDKNNVSPVEASLIGTPVADAKRPVEILRTVHSFDPCIACGVH'}

In [5]:
#check
#chain distance may give you some insight on chain's homology
aligner.chain_distance


{'A': {'S': 0.45503, 'L': 0.48148},
 'B': {'S': 0.3427, 'L': 0.48238},
 'C': {'S': 0.3427, 'L': 0.48238},
 'D': {'S': 0.46629, 'L': 0.37453},
 'E': {'S': 0.46629, 'L': 0.37453}}

In [6]:
#check the ligand list
aligner.ligands

[<Residue SF4 het=H_SF4 resseq=1001 icode= >,
 <Residue SF4 het=H_SF4 resseq=1002 icode= >,
 <Residue F3S het=H_F3S resseq=1003 icode= >,
 <Residue MG het=H_MG resseq=1005 icode= >,
 <Residue NFE het=H_NFE resseq=1004 icode= >]

In [17]:
#check
#residue in aligner.ligands is the same residue in aligner.reference
aligner.reference[0]['S'][('H_SF4', 1002, ' ')] is aligner.ligands[1]

True

In [18]:
#execute
#set the ligand of concern fro down stream work
aligner.set_ligand(aligner.reference[0]['S'][('H_F3S', 1003, ' ')])

In [25]:
#check
#aligner.neighbor save the  residue objects within 6 A of seolected ligand
#aligner.neighbor_chain save the chain id
aligner.neighbor
aligner.neighbor_chain


('S', 'L')

function 1: do not know the homology chain ?
run  aligner.transform() directly,
it will try all chain combination and return the combination with lowest pocket rmsd.
it will print: combination, number of hit residue,rmsd (full &pocket) 
and return: best chain combine; rmsd (full &pocket) 

then run aligner.save_align() to save the pdbs.

Warning!
you can see that since cmd.align will drop a few atoms in each alignment cycle,
wrong chain hit can have as nice behavior as the 'right one'
going to be debugged. 

In [19]:
aligner.transform()

('A', 'B')	12	(0.9076577425003052, 2.1527609825134277)	('A', 'C')	12	(0.9076577425003052, 2.1527609825134277)	('A', 'D')	12	(0.9076577425003052, 2.1527609825134277)	('A', 'E')	12	(0.9076577425003052, 2.1527609825134277)	('B', 'A')	20	(0.9076577425003052, 0.4204222559928894)	('B', 'C')	20	(0.9076577425003052, 0.4204222559928894)	('B', 'D')	20	(0.9076577425003052, 0.4204222559928894)	('B', 'E')	20	(0.9076577425003052, 0.4204222559928894)	('C', 'A')	20	(0.9076577425003052, 0.422482430934906)	('C', 'B')	20	(0.9076577425003052, 0.422482430934906)	('C', 'D')	20	(0.9076577425003052, 0.422482430934906)	('C', 'E')	20	(0.9076577425003052, 0.422482430934906)	('D', 'A')	20	(0.9076577425003052, 6.543617248535156)	('D', 'B')	20	(0.9076577425003052, 6.543617248535156)	('D', 'C')	20	(0.9076577425003052, 6.543617248535156)	('D', 'E')	20	(0.9076577425003052, 6.543617248535156)	('E', 'A')	20	(0.9076577425003052, 6.539333820343018)	('E', 'B')	20	(0.9076577425003052, 6.539333820343018)	('E', 'C')	20	(0.907

(('B', 'A'), (0.9076577425003052, 0.4204222559928894))

In [24]:
aligner.save_align()

(0.9076577425003052, 0.4204222559928894)

function 2: sure about the chain map?
set aligner.mapped_chain,and run aligner.save_align directly

In [28]:
aligner.mapped_chain=('B','D')
aligner.save_align(ligandfile='bdligand.pdb',
                    ref_pocketfile='bdaligned_pocket.pdb',
                    obj_pocketfile='bdmodel_pocket.pdb')

(0.9076577425003052, 0.4204222559928894)